In [4]:
import re
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
import argparse
import joblib

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.client import device_lib

In [2]:
# 1.데이터 로드
root_path = '/home/ubuntu/workspace/seunghoon/naver_movie_text_classification/LSTM/dataset/nsmc'

train_data = pd.read_table(os.path.join(root_path, 'ratings_train.txt'))
test_data = pd.read_table(os.path.join(root_path, 'ratings_test.txt'))

# 2.데이터 정제
train_data['document'].nunique(), train_data['label'].nunique() # 중복제거
train_data.drop_duplicates(subset=['document'], inplace=True) # document 열의 중복 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True) # 한글과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace('^ +', "", regex=True) # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how = 'any')

test_data['document'].nunique(), test_data['label'].nunique() # 중복 제거
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True) # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "", regex=True) # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거


# 3.토큰화
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
okt = Okt()

X_train = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)

X_test = []
for sentence in tqdm(test_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

# 4.정수 인코딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1

tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

# 5.빈 샘플 제거
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)

# 6.패딩
max_len = 30

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

100%|██████████| 48852/48852 [02:00<00:00, 404.55it/s]
/home/ubuntu/anaconda3/envs/seunghoon/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [22]:
pickle_lst = [train_data, test_data, tokenizer, X_train, X_test, y_train, y_test]
name_lst = ["train_data.pkl", "test_data.pkl", "tokenizer.pkl", "X_train.pkl", "X_test.pkl", "y_train.pkl", "y_test.pkl"]

In [24]:
for name, object_name in zip(name_lst, pickle_lst):
    with open(name, "wb")as f:
        pickle.dump(object_name, f)

In [23]:
root_path = '/home/ubuntu/workspace/seunghoon/naver_movie_text_classification/LSTM/dataset/nsmc'

train_data = pd.read_table(os.path.join(root_path, 'ratings_train.txt'))
test_data = pd.read_table(os.path.join(root_path, 'ratings_test.txt'))

# 2.데이터 정제
train_data['document'].nunique(), train_data['label'].nunique() # 중복제거
train_data.drop_duplicates(subset=['document'], inplace=True) # document 열의 중복 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True) # 한글과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace('^ +', "", regex=True) # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how = 'any')

test_data['document'].nunique(), test_data['label'].nunique() # 중복 제거
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True) # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "", regex=True) # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거

In [24]:
print(len(train_data))
print(len(test_data))
print(train_data[:1])
print(test_data[:1])

145393
48852
        id           document  label
0  9976970  아 더빙 진짜 짜증나네요 목소리      0
        id document  label
0  6270596      굳 ㅋ      1


In [25]:
def data_cleansing(dataset):
    dataset['document'].nunique(), dataset['label'].nunique() # 중복제거
    dataset.drop_duplicates(subset=['document'], inplace=True) # document 열의 중복 제거
    dataset['document'] = dataset['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True) # 한글과 공백을 제외하고 모두 제거
    dataset['document'] = dataset['document'].str.replace('^ +', "", regex=True) # white space 데이터를 empty value로 변경
    dataset['document'].replace('', np.nan, inplace=True)
    dataset = dataset.dropna(how = 'any')

In [26]:
root_path = '/home/ubuntu/workspace/seunghoon/naver_movie_text_classification/LSTM/dataset/nsmc'

train_data = pd.read_table(os.path.join(root_path, 'ratings_train.txt'))
test_data = pd.read_table(os.path.join(root_path, 'ratings_test.txt'))

In [27]:
data_cleansing(train_data)
data_cleansing(test_data)

In [28]:
print(len(train_data))
print(len(test_data))
print(train_data[:1])
print(test_data[:1])

146183
49158
        id           document  label
0  9976970  아 더빙 진짜 짜증나네요 목소리      0
        id document  label
0  6270596      굳 ㅋ      1


In [40]:
a_lst = [1,2,3]
b_lst = ['a','b','c']
c_lst = ['가','나','다']

In [41]:
object_lst = [a_lst, b_lst, c_lst]
name_lst = ["test1.pkl", "test2.pkl", "test3.pkl"]

for object_name, name in zip(object_lst, name_lst):
    joblib.dump(object_name, name)

In [58]:
import requests
import json
rslt = requests.post('http://localhost:1234/home', json.dumps('ㅋㅋ 응 안봐~')).text
print(json.loads(rslt))

64.74% 확률로 부정 리뷰입니다.

